Теорема, доказанная Джорджем Цыбенко в 1989 году, которая утверждает, что искусственная нейронная сеть прямой связи с одним скрытым слоем может аппроксимировать любую непрерывную функцию многих переменных с любой точностью. 

In [25]:
%matplotlib
import numpy as np
import sys
import math
import matplotlib.pyplot as plt

Using matplotlib backend: <object object at 0x0000017D59117720>


# Апроксимируемая функция

In [26]:
#Функция для аппроксимации
def ApproxFunc(x):
    return x**2
    #return np.sin(x)*10 + x**2

In [27]:
#интервал, на котором будем аппроксимировать функцию
pcenter = 0
prange = 10
step = 0.1
period = np.arange(pcenter-prange , pcenter+prange,step)
plt.plot(period,ApproxFunc(period))

# Реализация нейронной сети

In [28]:
class ApproximationNN(object):
    def __init__(self, learning_rate=0.1, input_nodes=1, hidden_nodes=5, output_nodes=1):
        input_nodes += 1
        self.weights_0_1 = np.random.normal(
            0.0, hidden_nodes ** -0.5, (hidden_nodes, input_nodes))
        self.weights_1_2 = np.random.normal(
            0.0, output_nodes ** -0.5, (output_nodes, hidden_nodes))
        self.sigmoid_mapper = np.vectorize(self.sigmoid)
        self.learning_rate = np.array([learning_rate])

    def set_lr(lr):
        self.learning_rate = np.array([lr])
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def predict(self, inputs):
        inputs = np.concatenate((inputs,[1]))
        inputs_1 = np.dot(self.weights_0_1, inputs)
        outputs_1 = self.sigmoid_mapper(inputs_1)
        
        inputs_2 = np.dot(self.weights_1_2, outputs_1)
        #outputs_2 = self.sigmoid_mapper(inputs_2)
        outputs_2 = inputs_2
        return outputs_2
    
    def train(self, inputs, expected_predict):
        inputs = np.concatenate((inputs,[1]))        
        inputs_1 = np.dot(self.weights_0_1, inputs)
        outputs_1 = self.sigmoid_mapper(inputs_1)
        
        inputs_2 = np.dot(self.weights_1_2, outputs_1)
        #outputs_2 = self.sigmoid_mapper(inputs_2)
        outputs_2 = inputs_2
        actual_predict = outputs_2[0]
        
        error_layer_2 = np.array([actual_predict - expected_predict])
        gradient_layer_2 = 1 # Здесь что-то я не так сделал. Нужно учитывать вес# actual_predict * (1 - actual_predict)
        weights_delta_layer_2 = error_layer_2 * gradient_layer_2  
        self.weights_1_2 -= (np.dot(weights_delta_layer_2, outputs_1.reshape(1, len(outputs_1)))) * self.learning_rate
        
        error_layer_1 = weights_delta_layer_2 * self.weights_1_2
        gradient_layer_1 = outputs_1 * (1 - outputs_1)
        weights_delta_layer_1 = error_layer_1 * gradient_layer_1
        self.weights_0_1 -= np.dot(inputs.reshape(len(inputs), 1), weights_delta_layer_1).T  * self.learning_rate
            


In [29]:
lr = 0.00005
myNN = ApproximationNN(learning_rate=lr, input_nodes=1,
                       hidden_nodes=50, output_nodes=1)


In [30]:
set_count = 100
rand_set = np.random.random(set_count)*2*prange-prange+pcenter
plt.plot(rand_set)

In [31]:
#Функция обучения
def Train():
    rand_set = np.random.random(set_count)*2*prange-prange+pcenter
    for x in range(0, set_count-1):
        myNN.train([rand_set[x]], ApproxFunc(rand_set[x]))

In [32]:
def MSE(y, Y):
    return np.mean((y-Y)**2)

In [33]:
epochs = 100
plt.ion()
for e in range(epochs):
    for tr in range(100):
        Train()
    plt.clf()
    plt.plot(period, ApproxFunc(period), 'blue')
    rand_set = np.random.random(set_count)*2*prange-prange+pcenter
    result_set=np.zeros(100)
    MSE=0
    for x in range(0, set_count-1):
        result_set[x] = myNN.predict([rand_set[x]])
        MSE+= (ApproxFunc(rand_set[x]) - result_set[x])**2
        plt.scatter(rand_set[x], result_set[x], c='red')
    plt.pause(1)    
    print(MSE/set_count)

864.8214914430728
906.9204890706487
567.7623897907531
339.6951009377171
276.3117785927636
264.46766769475255
111.57685503714993
128.96242185186887
44.31378484264674
49.440430899058626
37.46380612062475
24.173570218971868
19.699181515456836
17.995320917423115
18.782509799471505
17.43865984762461
13.056914514442393
8.2439817715827
8.890604659141127
10.52757468345332
4.471864836976186
7.88788458111945
4.557055129697579
8.982068370639832
4.889262009805817
3.703607930968767
3.324409243166347
4.527072128364025
3.7417963784788975
3.4259297544328104
3.4456798456456905


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\backends\_backend_tk.py", line 251, in idle_draw
    self.draw()
  File "c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\backends\backend_tkagg.py", line 10, in draw
    super().draw()
  File "c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\backends\backend_agg.py", line 405, in draw
    self.figure.draw(self.renderer)
  File "c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\artist.py", line 74, in draw_wrapper
    result = draw(artist, renderer

1.8798602902688664
2.5147576786007346
1.5866683368135233
2.443299478573538


In [ ]:
myNN.set_lr(0.000001)